## Initialization

In [1]:
import sys
#sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('I:/내 드라이브/backup/document/USC/Research/MCPC/Code/Codes')

#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

#import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
#from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

#import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

import scipy

import igraph 
import networkx as nx


import metric as met 
import embedding as embed   
import data_utils_ch as data_util

%load_ext autoreload
%autoreload 2
import importlib

import metric as met 
import embedding as embed   
import data_utils_ch as data_util
from subgraph import *


met=importlib.reload(met)
embed=importlib.reload(embed)
data_util=importlib.reload(data_util)

## Clean Version

In [32]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
data_names = ['Zhengmix8eq']
#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05,0.25, 0.5, 1, 1.5, 5]

for data in data_names: 
    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'
    data_for_plot = defaultdict(lambda: defaultdict(dict)) #nested dictionary to store data for plots

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)
    print('Number of nodes:',G.number_of_nodes())

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                ''' 
                Louvain on G (baseline)
                '''
                label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
                Num_Cluster_Louv_on_G = len(set(label_))

                '''
                Leiden on G (Baseline)
                '''
                label_, NMI_baseline_leiden, Purity_baseline_leiden = do_Leiden(G, G, res, 0, label)
                Num_Cluster_Leiden_on_G = len(set(label_))

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Louvain on the subgraph H 
                    '''
                    H_label, _, _ = do_Louvain(H, G, res, 0, label)
                    
                    Num_Cluster_Louv_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    '''
                    Plots
                    '''
                    #at index [res][FR_type] store the data for the plot
                    data_for_plot[idx][FR_type] = {'NMI_after_vote':NMI_List[-1], 'Purity_after_vote':Purity_List[-1], 'Louvain_NMI':NMI_, 
                                                    'Louvain_Purity':Purity_, 'Leiden_NMI':NMI_baseline_leiden, 'Leiden_Purity':Purity_baseline_leiden, 
                                                    'Num_Cluster_Louv_on_G':Num_Cluster_Louv_on_G, 'Num_Cluster_Leiden_on_G':Num_Cluster_Leiden_on_G, 
                                                    'Num_Cluster_Louv_on_H':Num_Cluster_Louv_on_H, 'percent_nodes':(selected_node_num*100 / G.number_of_nodes()),
                                                    }
                    
    plot_to_pdf(resolution_list, data_for_plot, k, True_num_of_clusters, pdf_name, data)
    
        
            

                                
                    

                    
                    



Zhengmix8eq 3994
Log transform done
(3994, 50)
Accuracy of  15 -NN graph is 0.792
59910
Number of nodes: 3994


## Compare top 20% subgraph and whole graph 

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''

        G, label= data_to_graph(data,survive)
        True_num_of_clusters = len(set(label))
        cluster_sizes = defaultdict(int)
        for node in G.nodes():
            cluster_sizes[label[node]]+=1

        G_ = G.copy()
 
        for node in G_.nodes():
            if not list(G_.successors(node)):
                #print(data, ' has a node without outgoing edges:',node)
                G_.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())

        data_directory = f'./Data'
        file_path = f'{data_directory}/compare_top_k.txt'
        os.makedirs(data_directory, exist_ok=True)

        with open(file_path, 'a') as f:
            f.write('-------------------------------------------------------------------------\n')
            f.write('Data: ' + data + '\n')

        for Num_hop in [1]:
            #List of FR_values
            FR_precalculated = []
            for fr_type in range(4): 
                '''
                Calculate FlowRank for the whole graph
                '''
                if Num_hop==0:
                    node2FR = calc_FlowRank(G_, fr_type, 5)
                else:
                    node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
                FR_precalculated.append(node2FR)

            for idx, res in enumerate([10]):
                for Mod_type in [0]:
                    # ''' 
                    # Louvain on G (baseline)
                    # '''
                    # label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
                    # #Num of cluster 
                    # Num_Cluster_Louv_on_G = len(set(label_))
                    '''
                    Leiden on G (Baseline)
                    '''
                    label_, NMI_baseline_leiden, Purity_baseline_leiden = do_Leiden(G, G, res, 0, label)
                    #NMI of the top 20% nodes
                    
                    
                    for FR_type in range(4):
                       
                        '''
                        Get induced subgraph of top k% nodes in FR value
                        '''
                        node2FR = FR_precalculated[FR_type]

                        node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                        
                        k_ = int(k*len(node_list))
                        top_nodes = node_list[:k_]
                        nodes_rest = node_list[k_:]
                        H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                        
                        label_compressed, G_label_compressed = get_compressed_labels(label, label_, top_nodes)
                        #calculate NMI 
                        NMI_on_G = NMI(label_compressed, G_label_compressed)
                        Purity_on_G = met.purity_score(label_compressed, G_label_compressed)

                        '''
                        label counts for balancedness calculation
                        '''
                        selected_labels_dict = defaultdict(int)
                        for node in H.nodes():
                            selected_labels_dict[label[node]]+=1
                        
                        '''
                        Louvain on the subgraph H 
                        '''
                        H_label, _, _ = do_Leiden(H, G, res, 0, label)
                        
                        label_compressed_, H_label_compressed = get_compressed_labels(label, H_label, top_nodes)
                        #calculate NMI
                        NMI_on_H = NMI(label_compressed_, H_label_compressed)
                        Purity_on_H = met.purity_score(label_compressed_, H_label_compressed)

                        with open(file_path, 'a') as f:
                            f.write('Resolution & FR_type: ' + str(res) + ' & ' + str(FR_tp[FR_type]) + '\n')
                            f.write('NMI on_G | H: ' + str(round(NMI_on_G,3)) + ' | ' + str(round(NMI_on_H,3)) + ' Purity: ' + str(round(Purity_on_G,2)) + ' | ' + str(round(Purity_on_H,2)) + '\n')


                        

        
    
        
            

                                
                    

                    
                    



ALM 10068
Log transform done
(10068, 50)
Accuracy of  15 -NN graph is 0.701
151020
Number of nodes: 10068
AMB 12832
Log transform done
(12832, 50)
Accuracy of  15 -NN graph is 0.762
192480
Number of nodes: 12832
Baron_Human 8569
Log transform done
(8569, 50)
Accuracy of  15 -NN graph is 0.972
128535
Number of nodes: 8569
Baron_Mouse 1886
Log transform done
(1886, 50)
Accuracy of  15 -NN graph is 0.954
28290
Number of nodes: 1886
Muraro 2122
Log transform done
(2122, 50)
Accuracy of  15 -NN graph is 0.951
31830
Number of nodes: 2122
Segerstolpe 2133
Log transform done
(2133, 50)
Accuracy of  15 -NN graph is 0.932
31995
Number of nodes: 2133
Tcell-medicine 5759
Log transform done
(5759, 50)
Accuracy of  15 -NN graph is 0.737
86385
Number of nodes: 5759
TM 54865
Log transform done
(54865, 50)
Accuracy of  15 -NN graph is 0.949
822975
Number of nodes: 54865
VISP 15413
Log transform done
(15413, 50)
Accuracy of  15 -NN graph is 0.711
231195
Number of nodes: 15413
Xin 1449
Log transform done